In [2]:
# Import pandas
import pandas as pd

# Assign spreadsheet filename to `file`
file = 'Все глобалы за существование ЧБ.xlsx'

# Load spreadsheet
xl = pd.ExcelFile(file)

In [3]:
# Print the sheet names
sheet_names_list = xl.sheet_names
df = None
df_current = None

for i in range(len(sheet_names_list)):
    # Load a sheet into a DataFrame by name: df1
    df = xl.parse(sheet_names_list[i])
    df = df.loc[:,['id','mesTExt','type','dialogid', 'Mdate','AgentEmail']]
    df = df.drop_duplicates()
    #df.head()

    df['Mdate'] = pd.to_datetime(df['Mdate'],format='%Y-%m-%d')
    df['Mdate'] = df['Mdate'].dt.date

    df_filter = df['type'].isin([1])
    df1 = df[df_filter]

    a = df1.groupby(['dialogid', 'Mdate'])['mesTExt'].transform(lambda answer : " ".join(str(x) for x in answer))
    df12= a.to_frame()

    df4 = pd.concat([df12, df1.loc[:,['dialogid','Mdate']]], axis=1)
    df4 = df4.drop_duplicates()

    df_filter = df['type'].isin([2])
    df5 = df[df_filter]
    df5 = df5.loc[:,['dialogid', 'Mdate','AgentEmail']]
    df5 = df5.drop_duplicates()

    df_req=pd.merge(df4, df5, on=['dialogid', 'Mdate'], how='inner')
    if i>0:
        df = pd.concat([df_req, df_current], axis=0, ignore_index=True)
    df_current = df_req
    
df.to_csv("Заявки в чатбот.csv", encoding = 'utf-16', sep=",", index=False)   


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6641 entries, 0 to 6640
Data columns (total 4 columns):
mesTExt       6641 non-null object
dialogid      6641 non-null int64
Mdate         6641 non-null object
AgentEmail    6641 non-null object
dtypes: int64(1), object(3)
memory usage: 207.6+ KB


In [5]:
df.head()

,mesTExt,dialogid,Mdate,AgentEmail
0,"Добрый день, не могу подключиться к удалённом...",13556941,2020-04-01,sergeev.d@samberi.com
1,"Добрый день, не могу подключиться к удалённом...",13556941,2020-04-01,c2d_control@dv-nevada.ru
2,"Добрый день, не могу подключиться к удалённом...",13556941,2020-04-01,c2d_centoffice_sa@dv-nevada.ru
3,"Добрый день, не могу подключиться к удалённом...",13556941,2020-04-01,telizhenko.vs@dv-nevada.ru
4,"После ""Открыть"" окно уходит и никаких действий...",6003524,2020-04-01,solovev.sv@dv-nevada.ru


In [6]:
import re

deleted_symols = '[\\\\\'[\]!"$%&()*+,-./:;<=>?@^_`{|}~«»\n]'
func = lambda text : re.sub(deleted_symols, ' ', text)
df["without_punctuation_text"] = df["mesTExt"].apply(func)
df.head()

,mesTExt,dialogid,Mdate,AgentEmail,without_punctuation_text
0,"Добрый день, не могу подключиться к удалённом...",13556941,2020-04-01,sergeev.d@samberi.com,Добрый день не могу подключиться к удалённом...
1,"Добрый день, не могу подключиться к удалённом...",13556941,2020-04-01,c2d_control@dv-nevada.ru,Добрый день не могу подключиться к удалённом...
2,"Добрый день, не могу подключиться к удалённом...",13556941,2020-04-01,c2d_centoffice_sa@dv-nevada.ru,Добрый день не могу подключиться к удалённом...
3,"Добрый день, не могу подключиться к удалённом...",13556941,2020-04-01,telizhenko.vs@dv-nevada.ru,Добрый день не могу подключиться к удалённом...
4,"После ""Открыть"" окно уходит и никаких действий...",6003524,2020-04-01,solovev.sv@dv-nevada.ru,После Открыть окно уходит и никаких действий...


In [7]:
func = lambda text : ' '.join([elem for elem in text.split(' ') if elem.isdigit() == False])   
df['without_numbers_text'] = df["without_punctuation_text"].apply(func)
df.head()

,mesTExt,dialogid,Mdate,AgentEmail,without_punctuation_text,without_numbers_text
0,"Добрый день, не могу подключиться к удалённом...",13556941,2020-04-01,sergeev.d@samberi.com,Добрый день не могу подключиться к удалённом...,Добрый день не могу подключиться к удалённом...
1,"Добрый день, не могу подключиться к удалённом...",13556941,2020-04-01,c2d_control@dv-nevada.ru,Добрый день не могу подключиться к удалённом...,Добрый день не могу подключиться к удалённом...
2,"Добрый день, не могу подключиться к удалённом...",13556941,2020-04-01,c2d_centoffice_sa@dv-nevada.ru,Добрый день не могу подключиться к удалённом...,Добрый день не могу подключиться к удалённом...
3,"Добрый день, не могу подключиться к удалённом...",13556941,2020-04-01,telizhenko.vs@dv-nevada.ru,Добрый день не могу подключиться к удалённом...,Добрый день не могу подключиться к удалённом...
4,"После ""Открыть"" окно уходит и никаких действий...",6003524,2020-04-01,solovev.sv@dv-nevada.ru,После Открыть окно уходит и никаких действий...,После Открыть окно уходит и никаких действий...


In [8]:
df["tokenized_text"] = df['without_numbers_text']
df = df.drop(["without_numbers_text", "without_punctuation_text"], axis=1)
df.head()

,mesTExt,dialogid,Mdate,AgentEmail,tokenized_text
0,"Добрый день, не могу подключиться к удалённом...",13556941,2020-04-01,sergeev.d@samberi.com,Добрый день не могу подключиться к удалённом...
1,"Добрый день, не могу подключиться к удалённом...",13556941,2020-04-01,c2d_control@dv-nevada.ru,Добрый день не могу подключиться к удалённом...
2,"Добрый день, не могу подключиться к удалённом...",13556941,2020-04-01,c2d_centoffice_sa@dv-nevada.ru,Добрый день не могу подключиться к удалённом...
3,"Добрый день, не могу подключиться к удалённом...",13556941,2020-04-01,telizhenko.vs@dv-nevada.ru,Добрый день не могу подключиться к удалённом...
4,"После ""Открыть"" окно уходит и никаких действий...",6003524,2020-04-01,solovev.sv@dv-nevada.ru,После Открыть окно уходит и никаких действий...


In [9]:
n_words = len(set(" ".join(df["tokenized_text"]).split()))
n_words

14689

In [10]:
df["lowercase_text"] = df["tokenized_text"].apply(lambda text : text.lower())
df.head()

,mesTExt,dialogid,Mdate,AgentEmail,tokenized_text,lowercase_text
0,"Добрый день, не могу подключиться к удалённом...",13556941,2020-04-01,sergeev.d@samberi.com,Добрый день не могу подключиться к удалённом...,добрый день не могу подключиться к удалённом...
1,"Добрый день, не могу подключиться к удалённом...",13556941,2020-04-01,c2d_control@dv-nevada.ru,Добрый день не могу подключиться к удалённом...,добрый день не могу подключиться к удалённом...
2,"Добрый день, не могу подключиться к удалённом...",13556941,2020-04-01,c2d_centoffice_sa@dv-nevada.ru,Добрый день не могу подключиться к удалённом...,добрый день не могу подключиться к удалённом...
3,"Добрый день, не могу подключиться к удалённом...",13556941,2020-04-01,telizhenko.vs@dv-nevada.ru,Добрый день не могу подключиться к удалённом...,добрый день не могу подключиться к удалённом...
4,"После ""Открыть"" окно уходит и никаких действий...",6003524,2020-04-01,solovev.sv@dv-nevada.ru,После Открыть окно уходит и никаких действий...,после открыть окно уходит и никаких действий...


In [12]:
!pip install pymorphy2

In [13]:
df["lemmatized_text"] = df["lowercase_text"]

In [14]:
def to_lemmatize2(df):
    all_word_str = " ".join(df["lowercase_text"])
    all_word_list = all_word_str.split()
    all_unique_word = Series(all_word_list).unique()
    lemmatized_word_dict = {}
    lemmatizer = MorphAnalyzer()
    for word in all_unique_word:
        lemmatized_word_dict[word] = lemmatizer.normal_forms(word)[0]
    lemm_func = lambda text: ' '.join([lemmatized_word_dict[word] for word in text.split()])
    df['lemmatized_text'] = df['lowercase_text'].apply(lemm_func)
    return df, all_unique_word

In [17]:
%%time
from pandas import DataFrame, Series
from pymorphy2 import MorphAnalyzer

df, _ = to_lemmatize2(df)

Wall time: 3.99 s


In [18]:
df.head()

,mesTExt,dialogid,Mdate,AgentEmail,tokenized_text,lowercase_text,lemmatized_text
0,"Добрый день, не могу подключиться к удалённом...",13556941,2020-04-01,sergeev.d@samberi.com,Добрый день не могу подключиться к удалённом...,добрый день не могу подключиться к удалённом...,добрый день не мочь подключиться к удалённый р...
1,"Добрый день, не могу подключиться к удалённом...",13556941,2020-04-01,c2d_control@dv-nevada.ru,Добрый день не могу подключиться к удалённом...,добрый день не могу подключиться к удалённом...,добрый день не мочь подключиться к удалённый р...
2,"Добрый день, не могу подключиться к удалённом...",13556941,2020-04-01,c2d_centoffice_sa@dv-nevada.ru,Добрый день не могу подключиться к удалённом...,добрый день не могу подключиться к удалённом...,добрый день не мочь подключиться к удалённый р...
3,"Добрый день, не могу подключиться к удалённом...",13556941,2020-04-01,telizhenko.vs@dv-nevada.ru,Добрый день не могу подключиться к удалённом...,добрый день не могу подключиться к удалённом...,добрый день не мочь подключиться к удалённый р...
4,"После ""Открыть"" окно уходит и никаких действий...",6003524,2020-04-01,solovev.sv@dv-nevada.ru,После Открыть окно уходит и никаких действий...,после открыть окно уходит и никаких действий...,после открыть окно уходить и никакой действие ...
